In [1]:
# import libraries
import os 

import numpy as np

# to handle datasets
import pandas as pd

# to iterate
import itertools

# scale data
from sklearn.preprocessing import StandardScaler

# to build the model
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

# to evaluate the models
from sklearn.metrics import mean_absolute_error, median_absolute_error, r2_score

In [2]:
# model evaluation 
def mae(df, col_name):
    """
    Function to calculate MAE.
    """
    mae = mean_absolute_error(df['true'].values, df['predicted'].values)
    
    return pd.Series({col_name: mae})

In [3]:
# define variables
experiment = "experiment_2-BS6"

models = {"random_forest": RandomForestRegressor(random_state=1), "knn": KNeighborsRegressor(), "svm": SVR()}
target = "log_precision"

if experiment == "experiment_1":
    suites = ['cec2013', 'cec2014', 'cec2015', 'cec2017', 'bbob']
    algorithms = ["DE", "RealSpacePSO", "CMA"]
    budget = 100000
    index_col = ["f_id", "i_id"]

elif experiment == "experiment_2":
    suites = ["BS1", "BS2", "BS3", "BS4", "BS5"]
    budget = 10000
    algorithms = ["diagCMA", "DE", "PSO"]
    index_col = ["f0", "f1", "alpha"]
    
elif experiment == "experiment_2-BS6":
    suites = ["BS1", "BS2", "BS3", "BS4", "BS5", "BS6"]
    budget = 10000
    algorithms = ["diagCMA", "DE", "PSO"]
    index_col = ["f0", "f1", "alpha"]

In [4]:
# define directories
data_folder= f"Data/preprocessed/{experiment}"

results_folder = f"Results/{experiment}"
os.makedirs(results_folder, exist_ok=True)

In [5]:
# create results placeholder
model_performance = pd.DataFrame()


for model_name, model in models.items():
    print(f"model_name: {model_name}")\

    for algorithm_name in algorithms: 
        print(f"algorithm_name: {algorithm_name}")

        for train_suite, test_suite in list(itertools.permutations(suites, 2)):
            print(f"train: {train_suite}, test: {test_suite}")

            ## load data
            train_title = f"suite={train_suite}_algorithm_name={algorithm_name}_budget={budget}_dataset.csv"
            if experiment == "experiment_2-BS6" and train_suite == "BS6":
                data_folder = f"Data/preprocessed/experiment_2-BS6"
            else: 
                data_folder = f"Data/preprocessed/experiment_2"
            train = pd.read_csv(f"{data_folder}/{train_title}", index_col = index_col)
            if experiment == "experiment_2-BS6" and test_suite == "BS6":
                data_folder = f"Data/preprocessed/experiment_2-BS6"
            else: 
                data_folder = f"Data/preprocessed/experiment_2"
                
            test_title = f"suite={test_suite}_algorithm_name={algorithm_name}_budget={budget}_dataset.csv"
            test = pd.read_csv(f"{data_folder}/{test_title}", index_col = index_col)

            ## split on target and features
            y_train = train["precision"]  # target
            y_test = test["precision"] # target

            # calculate log precision of the 
            if target == "log_precision":

                y_train = np.log10(y_train + 1) # target
                y_test = np.log10(y_test + 1) # target

            X_train = train.drop("precision", axis=1) # data
            X_test = test.drop("precision", axis=1) # data

            print("preview X_train: ")
            print(X_train.head(1))
            print(X_train.shape)


            ## preprocess data
            scaler = StandardScaler()
            scaler = scaler.fit(X_train)

            X_train = pd.DataFrame(scaler.transform(X_train), index = X_train.index, columns=X_train.columns.tolist())
            X_test = pd.DataFrame(scaler.transform(X_test), index = X_test.index, columns=X_test.columns.tolist())
            
            print("preview X_train preprocessed: ")
            print(X_train.head(1))

            ## train model
            model.fit(X_train, y_train.values)

            ## make predictions
            predictions_train = model.predict(X_train)
            predictions_test = model.predict(X_test)


            ## model perfromance
            model_performance = model_performance.append({"target": target,
                                                          "model_name": model_name,
                                                          "algorithm_name": algorithm_name,
                                                          "train_suite": train_suite,
                                                          "test_suite": test_suite

                         , "mae_train": mean_absolute_error(y_train.values, predictions_train)
                         , "mdae_train": median_absolute_error(y_train.values, predictions_train)
                         , "r2_train": r2_score(y_train.values, predictions_train)

                         , "mae_test": mean_absolute_error(y_test.values, predictions_test)
                         , "mdae_test": median_absolute_error(y_test.values, predictions_test)
                         , "r2_test": r2_score(y_test.values, predictions_test)}
                         , ignore_index=True)


            ## concat model predictions
            model_predictions = pd.DataFrame(y_test.values, columns=["true"], index=X_test.index)
            model_predictions["predicted"] = predictions_test

            model_predictions.reset_index().to_csv(f"{results_folder}/predictions_train={train_suite}_test_={test_suite}_algorithm={algorithm_name}_model={model_name}.csv"
                                     , index=False)

model_name: random_forest
algorithm_name: diagCMA
train: BS1, test: BS2
preview X_train: 
                                                      ela_distr.skewness  \
f0                f1                        alpha                          
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461            1.123652   

                                                      ela_meta.lin_simple.adjr2  \
f0                f1                        alpha                                 
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461                    0.00656   

                                                      ela_meta.lin_simple.intercept  \
f0                f1                        alpha                                     
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461                     -43.664781   

                                                      ela_meta.lin_simple.coef_max  \
f0                f1                        alpha                                  

C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,
C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,


train: BS1, test: BS4
preview X_train: 
                                                      ela_distr.skewness  \
f0                f1                        alpha                          
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461            1.123652   

                                                      ela_meta.lin_simple.adjr2  \
f0                f1                        alpha                                 
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461                    0.00656   

                                                      ela_meta.lin_simple.intercept  \
f0                f1                        alpha                                     
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461                     -43.664781   

                                                      ela_meta.lin_simple.coef_max  \
f0                f1                        alpha                                    
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.8

C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,
C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,


train: BS1, test: BS6
preview X_train: 
                                                      ela_distr.skewness  \
f0                f1                        alpha                          
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461            1.123652   

                                                      ela_meta.lin_simple.adjr2  \
f0                f1                        alpha                                 
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461                    0.00656   

                                                      ela_meta.lin_simple.intercept  \
f0                f1                        alpha                                     
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461                     -43.664781   

                                                      ela_meta.lin_simple.coef_max  \
f0                f1                        alpha                                    
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.8

...

C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,
C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,
C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,
C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  

preview X_train preprocessed: 
                                                      ela_distr.skewness  \
f0                f1                        alpha                          
bbob_f019_i01_d05 shifted-bbob_f004_i01_d05 0.002221            0.415709   

                                                      ela_meta.lin_simple.adjr2  \
f0                f1                        alpha                                 
bbob_f019_i01_d05 shifted-bbob_f004_i01_d05 0.002221                  -0.868319   

                                                      ela_meta.lin_simple.intercept  \
f0                f1                        alpha                                     
bbob_f019_i01_d05 shifted-bbob_f004_i01_d05 0.002221                      -0.136403   

                                                      ela_meta.lin_simple.coef_max  \
f0                f1                        alpha                                    
bbob_f019_i01_d05 shifted-bbob_f004_i01_d05 0.002221    

C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,
C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,
C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,
C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  

train: BS6, test: BS2
preview X_train: 
                                                      ela_distr.skewness  \
f0                f1                        alpha                          
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461            0.025004   

                                                      ela_meta.lin_simple.adjr2  \
f0                f1                        alpha                                 
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461                   0.000146   

                                                      ela_meta.lin_simple.intercept  \
f0                f1                        alpha                                     
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.895461                      -0.971645   

                                                      ela_meta.lin_simple.coef_max  \
f0                f1                        alpha                                    
bbob_f004_i01_d05 shifted-bbob_f003_i01_d05 0.8

C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,
C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,
C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model_performance = model_performance.append({"target": target,
C:\Users\anani\AppData\Local\Temp\ipykernel_6892\1150169877.py:66: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  

In [6]:
# save model average perfroamnce              
model_performance.to_csv(f"{results_folder}/performance.csv"
                        , index=False)
model_performance.head()

,target,model_name,algorithm_name,train_suite,test_suite,mae_train,mdae_train,r2_train,mae_test,mdae_test,r2_test
0,log_precision,random_forest,diagCMA,BS1,BS2,0.095831,0.06575,0.904696,0.151179,0.089669,0.750426
1,log_precision,random_forest,diagCMA,BS1,BS3,0.095831,0.06575,0.904696,0.190729,0.112051,0.611115
2,log_precision,random_forest,diagCMA,BS1,BS4,0.095831,0.06575,0.904696,0.190626,0.117765,0.691892
3,log_precision,random_forest,diagCMA,BS1,BS5,0.095831,0.06575,0.904696,0.200708,0.110893,0.637074
4,log_precision,random_forest,diagCMA,BS1,BS6,0.095831,0.06575,0.904696,0.367093,0.318307,-0.009937


In [7]:
model_performance.shape

(270, 11)

In [8]:
# save
model_predictions.head()

,,,true,predicted
f0,f1,alpha,,
bbob_f022_i01_d05,shifted-bbob_f001_i01_d05,0.848012,1.220144,1.354079
bbob_f016_i01_d05,shifted-bbob_f003_i01_d05,0.783305,1.660559,1.354079
bbob_f003_i01_d05,shifted-bbob_f016_i01_d05,0.423922,1.728569,1.354079
bbob_f016_i01_d05,shifted-bbob_f014_i01_d05,0.994975,0.822862,1.354079
bbob_f019_i01_d05,shifted-bbob_f023_i01_d05,0.526235,0.830575,1.354079


In [9]:
model_predictions.shape

(53, 2)